FederatedScope  is a flexible FL framework, which enables users to implement complex FL algorithms simply and intuitively. In this tutorial, we will show how to implement diverse personalized FL algorithms.

## Background

In an FL course, multiple clients aim to cooperatively learn models without directly sharing their private data. As a result, these clients can be arbitrarily different in terms of their underlying **data distribution** and **system resources** such as computational power and communication width.

- On one hand, the data quantity skew, feature distribution skew,  label distribution skew, and temporal skew are pervasive in real-world applications as different users generate the data with different usage manners.
  Simply applying the shared global model for all participants might lead to sub-optimal performance.
- On the other hand, the participation degrees of different FL participants can be diverse due to their different hardware capabilities and network conditions.

It is challenging to make full use of local data considering such systematical heterogeneity. As a natural and effective approach to address these challenges, personalization gains increasing attention in recent years. Personalized FL (pFL) raises strong demand for various customized FL implementation, e.g., the personalization may exist in

- Model objects, optimizers and hyper-parameters
- Model sub-modules
- Client-end behaviors such as regularization and multi-model interaction
- Server-end behaviors such as model interpolation

We have implemented several implementations for state-of-the-art (SOTA) pFL methods to meet the above requirements. We will demonstrate the FedBN in this playground. More examples can be found in our [tutorial](https://federatedscope.io/docs/pfl/), in which we show how powerful and flexible the FederatedScope framework to implement pFL extensions.


## The necessity of personalization: A multi-task example

### Setting
Federated learning can be well modeled by multi-task learning, that is, viewing each client's learning as a subtask. A common multi-task learning scenario is that individual subtasks use different and non-overlapping features and labels. 
Although the features and labels are superficially different, the ability of sub-task models can be migrated and enhanced by the federated process.

Note that when the features and labels have different shape, it is necessary to personalize the input and output layers for the aggregation compatibility.
Here we use a multi-task graph dataset as an example since the dataset has a relatively small data size and we can check the performance quickly. 
For more larger datasets, please refer examples in `scripts/personalization_exp_scripts`.

### Configuration
**FederatedScope** organizes the configuration through an extension of `yacs.config.cfgNode`. Please refer to our [official documentation](https://federatedscope.io/docs/own-case/) for specific instructions on how to configure the initial `global_cfg` and customize your own configuration. By default, we provide built-in personalization-related configurations in `federatedscope/core/configs/`.

In [ ]:
from federatedscope.core.configs.config import global_cfg
import torch

pfl_cfg = global_cfg.clone()
pfl_cfg.merge_from_file("FederatedScope/federatedscope/gfl/baseline/fedavg_gnn_minibatch_on_multi_task.yaml")
pfl_cfg.use_gpu = torch.cuda.is_available()

print(pfl_cfg.personalization)

As you can see, we make the input and output layer as local personalized parameters by simply configuration.

### 1. Prepare dataset and model
Now, let's load the graph multi-task dataset and check the GIN model.

In [ ]:
from federatedscope.core.auxiliaries.data_builder import get_data

data, modified_cfg = get_data(pfl_cfg.clone())
pfl_cfg.merge_from_other_cfg(modified_cfg)
print(f"=====The data config is =====\n{pfl_cfg.data}\n")
print(f"=====The data meta-info is =====\n{data}\n")
print(f"=====The train data at client 1 is =====\n{data[1]['train'].dataset}\n")

In [ ]:
print(f"=====The model config is =====\n{pfl_cfg.model}\n")

##  2. Check the performance w/o FedBN
Now, let's check the FL performance from FedAvg. We first check several task-specific configuration speficied by our yaml:

In [ ]:
print(f"=====Evaluation related configs=====\n{pfl_cfg.eval}\n")
print(f"=====Federated setting related configs=====\n{pfl_cfg.federate}\n")

Then we can see that we only maintain local paramters of pre-layer and last-layer to handle multi-task case:

In [ ]:
print(pfl_cfg.personalization) 

Now let's run our API to see the performance of FedAvg! This will take a few minutes.

In [ ]:
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger

update_logger(pfl_cfg)
setup_seed(pfl_cfg.seed)

Fed_runner = FedRunner(data=data,
                       server_class=get_server_cls(pfl_cfg),
                       client_class=get_client_cls(pfl_cfg),
                       config=pfl_cfg.clone())
Fed_runner.run()

We finally log the best results based on the validation datasets. 
You can see that the FedAvg gains 0.615 acc when using weighted summarizaion manner. Besides, we find that the std of test accucary is 0.140, and the top and bottom acc deciles are 0.86 and 0.465 respectively.

##  2. Check the performance w/ FedBN
Now, lets use the FedBN and see the performance change!

In [ ]:
print(f"=====[Before] Federated personalization related configs=====\n{pfl_cfg.personalization}\n")

pfl_cfg.merge_from_file("FederatedScope/scripts/personalization_exp_scripts/fedbn/fedbn_gnn_minibatch_on_multi_task.yaml")
pfl_cfg.use_gpu = torch.cuda.is_available()

print(f"=====[After] Federated personalization related configs=====\n{pfl_cfg.personalization}\n")

The modification here is to filter the `norms` parameters. Let's run the API to check the performance of FedBN!

In [ ]:
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


Fed_runner = FedRunner(data=data,
                       server_class=get_server_cls(pfl_cfg),
                       client_class=get_client_cls(pfl_cfg),
                       config=pfl_cfg.clone())
Fed_runner.run()

We can find that simple personalized method achieves 0.741 acc when using weighted summarizaion manner. Besides, the std of test accucary is 0.138, and the top and bottom acc deciles are 0.927 and 0.514 respectively. 

Compared with FedAvg, we gain significant accuracy improvements (12.6%)! The fairness-related metrics are also improved: the standard deviation, the top and bottom deciles are improved with a ratio 0.2%, 6.7%, and 4.9% respectively.

Try other pFL algorithms from our [tutorial](https://federatedscope.io/docs/pfl/), and welcome to [contribute](https://federatedscope.io/docs/contributor/) more pFL algorithms!